<a href="https://colab.research.google.com/github/s2ul2/bitamin/blob/main/6week_WordEmbedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from tensorflow.keras.preprocessing.text import text_to_word_sequence

text = "해보지 않으면 해낼 수 없다"

result = text_to_word_sequence(text)
print("원문 : \n", text)
print("\n토큰화 : \n", result)

원문 : 
 해보지 않으면 해낼 수 없다

토큰화 : 
 ['해보지', '않으면', '해낼', '수', '없다']


# FastText & Glove실습

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [7]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [8]:
import re
import urllib.request
import zipfile
from lxml import etree
from nltk.tokenize import word_tokenize, sent_tokenize

urllib.request.urlretrieve("https://raw.githubusercontent.com/ukairia777/tensorflow-nlp-tutorial/main/09.%20Word%20Embedding/dataset/ted_en-20160408.xml", filename="ted_en-20160408.xml")
targetXML = open('ted_en-20160408.xml', 'r', encoding='UTF8')
target_text = etree.parse(targetXML)
parse_text = '\n'.join(target_text.xpath('//content/text()'))
content_text = re.sub(r'\([^)]*\)', '', parse_text)
sent_text = sent_tokenize(content_text)
normalized_text = []

for string in sent_text:
    tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
    normalized_text.append(tokens)

result = [word_tokenize(sentence) for sentence in normalized_text]


In [14]:
from gensim.models import FastText
model = FastText(result, size=100,  window = 5, min_count = 5, workers = 4, sg = 1)

In [15]:
model.wv['man']

array([ 2.4593151e-01,  7.7986467e-01, -3.4457189e-01, -3.7506098e-01,
        2.9399818e-01,  2.4272403e-01,  7.9590857e-02,  3.2319111e-01,
       -1.9689107e-01,  3.4416279e-01, -6.4711911e-03, -8.2396306e-02,
       -4.9597841e-02,  1.9377755e-02, -1.9609740e-01,  2.1611449e-01,
       -1.7658825e-01, -4.2087179e-01,  2.4283302e-01,  2.0130980e-01,
        8.8523529e-02,  4.8056751e-01,  1.3011719e-02, -7.9116803e-01,
        3.8578090e-01, -5.8579862e-02, -3.4767812e-01,  1.5521574e-01,
       -4.9736938e-01, -9.0103149e-03, -8.0594793e-02,  3.0712556e-02,
        1.9097683e-01, -3.1651345e-03,  3.6152270e-01, -1.3389663e-01,
        2.4202943e-02, -2.8888533e-01, -8.7604523e-02, -6.4251252e-04,
       -1.0246045e-01, -1.6434068e-02,  3.0469134e-01, -1.1592449e-01,
        5.8694261e-01,  3.1241354e-01, -1.9383383e-01,  1.1755972e-01,
        9.0927482e-02, -2.5490437e-02,  1.8153940e-01, -3.5813433e-01,
       -1.2593733e-01, -7.7940039e-02,  5.4977107e-01, -2.6232696e-01,
      

In [17]:
model.wv.most_similar('electrofishing')

[('electrolux', 0.8084871172904968),
 ('electro', 0.7906161546707153),
 ('electrolyte', 0.7892001867294312),
 ('electric', 0.7782672643661499),
 ('electroshock', 0.7672708630561829),
 ('electrochemical', 0.7660350799560547),
 ('gastric', 0.7594130039215088),
 ('overfishing', 0.7530572414398193),
 ('airbus', 0.7520583868026733),
 ('fishing', 0.7506319284439087)]

In [18]:
pip install glove_python_binary

     |████████████████████████████████| 948 kB 4.1 MB/s 


In [19]:
from glove import Corpus, Glove

corpus = Corpus()

corpus.fit(result, window = 5)
glove = Glove(no_components = 100, learning_rate = 0.05)

# 학습
glove.fit(corpus.matrix, epochs = 20, no_threads = 4, verbose = True)
glove.add_dictionary(corpus.dictionary)

Performing 20 training epochs with 4 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19


In [20]:
glove.most_similar('man')

[('woman', 0.9605496647684298),
 ('guy', 0.8874339733405103),
 ('girl', 0.8511348356837268),
 ('young', 0.8458955598061602)]

In [21]:
glove.most_similar('water')

[('clean', 0.8500364034508788),
 ('air', 0.8433729083604333),
 ('fresh', 0.8296182660902678),
 ('electricity', 0.8204649746906137)]

# 전이학습

In [23]:
import os
import sys

import numpy as np
import pandas as pd
from numpy import array
from numpy import asarray
from numpy import zeros

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import models, regularizers, layers, optimizers, losses, metrics
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Flatten
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence

import tensorflow as tf
tf.random.set_seed(1)
np.random.seed(1)

In [25]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [26]:
df = pd.read_csv('/content/drive/MyDrive/비타민/2학기/6week_WordEmbedding/data.csv')
df.head()

,paragraph,category
0,dishplace is located in sunnyvale downtown the...,food
1,service can be slower during busy hours but ou...,food
2,portions are huge both french toast and their ...,food
3,we started with apps going the chicken and waf...,food
4,the biscuits and gravy was too salty two peopl...,food


In [27]:
docs = df['paragraph'].tolist()
categories = df['category'].tolist()

In [31]:
t = Tokenizer()
t.fit_on_texts(docs)
vocab_size = len(t.word_index) + 1
encoded_docs = t.texts_to_sequences(docs)

def get_max_length(df):
    max_length = 0
    for row in df['paragraph']:
        if len(row.split(" ")) > max_length:
            max_length = len(row.split(" "))
    return max_length

max_length = get_max_length(df)
print (max_length)

padded_docs = pad_sequences(encoded_docs, maxlen = max_length, padding = 'post')

91


In [32]:
padded_docs[0]

array([162,  16, 163,   5, 164, 165,  56,  16, 166, 167,   1, 168,   9,
         6,  91,  25,  92,   2, 169,  40, 170, 171,  57,  26, 172,   3,
        12,  41,   2,  23,  58,   8, 173,  15,   4, 174,  10,  13,  59,
        93,  17,  12, 175, 176, 177, 178, 179,  94,  42,   1, 180,   2,
        95,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
      dtype=int32)

In [33]:
def category_encode(category):
    if category == 'food':
        return[1, 0]
    else:
        return [0, 1]

labels = array([category_encode(category) for category in categories])
print("first document's label : ", labels[0])

first document's label :  [1 0]


In [35]:
# Glove 다운

from urllib.request import urlretrieve, urlopen
import gzip
import zipfile

urlretrieve("http://nlp.stanford.edu/data/glove.6B.zip", filename = 'glove.6B.zip')
zf = zipfile.ZipFile('glove.6B.zip')
zf.extractall()
zf.close

<bound method ZipFile.close of <zipfile.ZipFile filename='glove.6B.zip' mode='r'>>

In [36]:
# 다운로드 받은 Glove 벡터 불러오기
embeddings_index = dict()
f = open('glove.6B.100d.txt', encoding = 'utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = asarray(values[1:], dtype = 'float32')
    embeddings_index[word] = coefs

f.close()

embedding_matrix = zeros((vocab_size, 100))
for word, i in t.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [39]:
model = Sequential()
e = Embedding(vocab_size, 100, weights = [embedding_matrix], input_length = max_length, trainable = False)
model.add(e)
model.add(Flatten())
model.add(Dense(2, activation = 'softmax'))

model. compile(loss = 'categorical_crossentropy',
               optimizer = 'adam',
               metrics = ['accuracy'])

model.fit(padded_docs, array(labels), epochs = 5, verbose = 1)

Epoch 1/5
1/1 [==============================] - 1s 791ms/step - loss: 0.7972 - accuracy: 0.4500
Epoch 2/5
1/1 [==============================] - 0s 34ms/step - loss: 0.3481 - accuracy: 0.9000
Epoch 3/5
1/1 [==============================] - 0s 12ms/step - loss: 0.1604 - accuracy: 1.0000
Epoch 4/5
1/1 [==============================] - 0s 13ms/step - loss: 0.0866 - accuracy: 1.0000
Epoch 5/5
1/1 [==============================] - 0s 17ms/step - loss: 0.0496 - accuracy: 1.0000


In [38]:
# 모델 생성
model = Sequential()
model.add(Embedding(vocab_size, 100, weights = [embedding_matrix], input_length = max_length, trainable = True))
model.add(Flatten())
model.add(Dense(2, activation = 'softmax'))

model.compile(loss = 'categorical_crossentropy',
              optimizer = 'adam',
              metrics = ['accuracy'])

model.fit(padded_docs, labels, epochs = 5, verbose = 1)

Epoch 1/5
1/1 [==============================] - 4s 4s/step - loss: 0.6478 - accuracy: 0.6000
Epoch 2/5
1/1 [==============================] - 0s 18ms/step - loss: 0.2867 - accuracy: 0.9000
Epoch 3/5
1/1 [==============================] - 0s 17ms/step - loss: 0.1236 - accuracy: 1.0000
Epoch 4/5
1/1 [==============================] - 0s 14ms/step - loss: 0.0680 - accuracy: 1.0000
Epoch 5/5
1/1 [==============================] - 0s 15ms/step - loss: 0.0410 - accuracy: 1.0000
